In [12]:
import pandas as pd
import json
from datetime import datetime
import s3fs
import numpy as np
import boto3
from datetime import datetime
import io

def Tweet_Api_Cleansed_Data():
    s3 = boto3.client('s3',aws_access_key_id='',aws_secret_access_key='') #provide your AWS Credentials
    obj = s3.get_object(Bucket = 'twitter-airflow-etl-data',Key = 'farmers-protest-tweets-2021-03-5.json')
    # json_data = obj['Body'].read().decode('utf-8')
    Tweet_File_Data = pd.read_json(obj['Body'].read().decode('utf-8'),lines = True)
    Tweet_File_Data = Tweet_File_Data[Tweet_File_Data['lang'] == 'en'][['id','user','renderedContent','date','media','likeCount','replyCount','retweetCount','quoteCount','quotedTweet','mentionedUsers','url']].fillna(np.nan)
    # Tweet_File_Data.fillna(np.nan,inplace = True)
    Tweet_File_Data['username'] = Tweet_File_Data['user'].apply(lambda x: x['username'] if 'username' in x else np.nan)
    Tweet_File_Data.loc[:,'verified'] = Tweet_File_Data['user'].apply(lambda x : x['verified'] if 'verified' in x else np.nan)
    Tweet_File_Data.loc[:,'accountCreated'] = Tweet_File_Data['user'].apply(lambda x : x['created'] if 'created' in x else np.nan)
    Tweet_File_Data['accountCreated'] = pd.to_datetime(Tweet_File_Data['accountCreated'])
    Tweet_File_Data.loc[:,'userUrl'] = Tweet_File_Data['user'].apply(lambda x : x['url'] if 'url' in x else np.nan)
    Tweet_File_Data.loc[:,'media'] = Tweet_File_Data[~Tweet_File_Data['media'].isna()]['media'].apply(lambda x : x[0]['fullUrl'] if 'fullUrl' in x[0] else x[0]['variants'][0]['url'] if 'url' in x[0]['variants'][0] else np.nan)
    Tweet_File_Data.loc[:,'quotedUrl']  = Tweet_File_Data[~Tweet_File_Data['quotedTweet'].isna()]['quotedTweet'].apply(lambda x : x['url'] if 'url' in x else np.nan)
    Tweet_File_Data.loc[:,'quotedUser'] = Tweet_File_Data[~Tweet_File_Data['quotedTweet'].isna()]['quotedTweet'].apply(lambda x : x['user']['username'] if 'username' in x['user'] else np.nan)
    Tweet_File_Data.loc[:,'quotedContent'] = Tweet_File_Data[~Tweet_File_Data['quotedTweet'].isna()]['quotedTweet'].apply(lambda x : x['content'] if 'content' in x else np.nan)
    Tweet_File_Data = Tweet_File_Data[['username','verified','accountCreated','userUrl','renderedContent','url','date','media','likeCount','replyCount','retweetCount','quotedUser','quotedContent','quotedUrl']]
    Tweet_File_Data.rename(columns = {'renderedContent':'tweetContent','url':'tweetUrl','date':'tweetPosted'},inplace = True)
    String_Data = io.StringIO()
    Tweet_File_Data.to_csv(String_Data,index = False)
    Twitter_Final_Data = String_Data.getvalue()
    # --export data to csv
    s3.put_object(Bucket = 'twitter-transformed-data',Key =  'Twitter_ETL_Data'+str(datetime.now())+'.csv',Body = Twitter_Final_Data)
    
    
    

In [14]:
Tweet_Api_Cleansed_Data()

C:\Users\polam\AppData\Local\Temp\ipykernel_6764\994265505.py:14: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  Tweet_File_Data = pd.read_json(obj['Body'].read().decode('utf-8'),lines = True)
